In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('pcafilewlabels.csv', header = None)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.555856,0.000025,0.000419,0.000038,-0.000716,-0.000726,0.001239,-0.000194,0.002798,0.012813,...,0.001206,0.000655,0.000001,-0.000071,-0.000121,-0.000157,-0.000213,-0.000554,p1,a01
1,-0.556310,0.000028,0.000418,0.000036,-0.000713,-0.000724,0.001234,-0.000199,0.002731,0.012509,...,0.001149,0.000652,-0.000043,-0.000076,-0.000117,-0.000103,-0.000173,-0.000490,p1,a01
2,-0.555609,0.000018,0.000419,0.000021,-0.000703,-0.000718,0.001231,-0.000156,0.002711,0.012504,...,0.000851,0.000605,-0.000239,-0.000013,-0.000219,-0.000058,-0.000412,-0.000616,p1,a01
3,-0.555990,0.000029,0.000417,0.000039,-0.000715,-0.000728,0.001247,-0.000215,0.002794,0.012732,...,0.001167,0.000664,-0.000019,-0.000037,-0.000138,-0.000130,-0.000105,-0.000490,p1,a01
4,-0.555658,0.000019,0.000425,0.000024,-0.000701,-0.000714,0.001223,-0.000171,0.002714,0.012477,...,0.001020,0.000639,-0.000154,-0.000130,-0.000097,-0.000048,-0.000214,-0.000560,p1,a01


In [8]:
X2 = df.iloc[:,30:32]
X2 = X2.apply(LabelEncoder().fit_transform)

In [9]:
X_2 = df.iloc[:,0:30].join(X2)
X_2.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.555856,0.000025,0.000419,0.000038,-0.000716,-0.000726,0.001239,-0.000194,0.002798,0.012813,...,0.001206,0.000655,0.000001,-0.000071,-0.000121,-0.000157,-0.000213,-0.000554,0,0
1,-0.556310,0.000028,0.000418,0.000036,-0.000713,-0.000724,0.001234,-0.000199,0.002731,0.012509,...,0.001149,0.000652,-0.000043,-0.000076,-0.000117,-0.000103,-0.000173,-0.000490,0,0
2,-0.555609,0.000018,0.000419,0.000021,-0.000703,-0.000718,0.001231,-0.000156,0.002711,0.012504,...,0.000851,0.000605,-0.000239,-0.000013,-0.000219,-0.000058,-0.000412,-0.000616,0,0
3,-0.555990,0.000029,0.000417,0.000039,-0.000715,-0.000728,0.001247,-0.000215,0.002794,0.012732,...,0.001167,0.000664,-0.000019,-0.000037,-0.000138,-0.000130,-0.000105,-0.000490,0,0
4,-0.555658,0.000019,0.000425,0.000024,-0.000701,-0.000714,0.001223,-0.000171,0.002714,0.012477,...,0.001020,0.000639,-0.000154,-0.000130,-0.000097,-0.000048,-0.000214,-0.000560,0,0


In [10]:
X_2[31].value_counts()

15    480
18    480
8     480
16    480
1     480
9     480
17    480
2     480
10    480
3     480
7     480
11    480
4     480
12    480
5     480
13    480
6     480
14    480
0     480
Name: 31, dtype: int64

In [14]:
X_2.iloc[:,0:30].shape

(9120, 30)

In [32]:
X = X_2.iloc[:,0:30].as_matrix()
y = X_2[31].as_matrix()

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 451)

In [34]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [35]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

(torch.Size([6840, 30]),
 torch.Size([2280, 30]),
 torch.Size([6840]),
 torch.Size([2280]))

In [36]:
train = torch.utils.data.TensorDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(train, batch_size=4,
                                     shuffle=True, num_workers=2)

In [63]:
x = X_2.iloc[:,0:30].as_matrix()
y = X_2[31].as_matrix()

In [64]:
x = torch.from_numpy(X)
y = torch.from_numpy(y)

In [66]:
training_samples = torch.utils.data.TensorDataset(X_train, y_train)
test_samples = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(training_samples, batch_size=64)
valid_loader = torch.utils.data.DataLoader(test_samples, batch_size=64)

In [74]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(30, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [75]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

TypeError: torch.addmm received an invalid combination of arguments - got (int, torch.FloatTensor, int, torch.DoubleTensor, torch.FloatTensor, out=torch.FloatTensor), but expected one of:
 * (torch.FloatTensor source, torch.FloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (torch.FloatTensor source, torch.SparseFloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (float beta, torch.FloatTensor source, torch.FloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (torch.FloatTensor source, float alpha, torch.FloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (float beta, torch.FloatTensor source, torch.SparseFloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (torch.FloatTensor source, float alpha, torch.SparseFloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTensor out)
 * (float beta, torch.FloatTensor source, float alpha, torch.FloatTensor mat1, torch.FloatTensor mat2, *, torch.FloatTen